In [1]:
import os
import re
import pickle
import pandas as pd
import numpy as np

from itertools import product
from collections import namedtuple, defaultdict
from collections.abc import Iterable
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt

# %matplotlib inline

Experiment = namedtuple('Experiment', ['d', 'k', 'imin', 'n', 't', 'imax'])

with open('results/results_data.pickle', 'rb') as handle:
    experiments = pickle.load(handle)
    
with open('results/control_data.pickle', 'rb') as handle:
    control = pickle.load(handle)
    
if not os.path.exists('graphs'):
    os.makedirs('graphs')
    
def filter_experiments(experiments, mask):
    # Mask is a namedtuple that has None for fields that are to be ignored
    mask_base = set(filter(lambda y: not isinstance(y[1], tuple), 
                           filter(lambda x: x[1], mask._asdict().items())))
    mask_permutes = set(filter(lambda y: isinstance(y[1], tuple),
                              filter(lambda x: x[1], mask._asdict().items())))
    
    # Delta is the permuting parameter expansions
    delta = list(map(lambda x: set(product((x[0],), x[1])), mask_permutes))
    permutations = [x.union(mask_base) for x in list(map(lambda x: set(x), product(*delta)))]
    
    # All of the experiments that match the filter(s)
    exps = list(map(lambda x: set(filter(lambda y: x <= set(y._asdict().items()), experiments)), permutations))
    
    # Collapse all of the experiments into a set and return it
    return reduce(lambda x, y: x.union(y), exps)

In [2]:
def filter_anomalies(data):
    mean = np.mean(data)
    std = np.std(data)
    
    return [x for x in data if (mean - 2 * std) < x < (mean + 2 * std)]

### Different recovery delay analysis

In [8]:
def recovery_delay(imin=16):
    filter_params = {
        'd': None, # d is the X axis here
        'k': (3, 4),
        'imin': imin,
        't': 'location',
        'imax': (11, ), 
        'n': 17
    }
    all_experiments = filter_experiments(experiments, Experiment(**filter_params))
    
    if not os.path.exists('graphs/mote_delay/'):
        os.makedirs('graphs/mote_delay')
    
    for i in filter_params['imax']:
        sub_filter = Experiment(d=None, k=None, imin=None, t='', imax=i, n=None)
        sub_experiments = list(filter_experiments(all_experiments, sub_filter))
        sub_experiments = sorted(sub_experiments, key=lambda x: (int(x.d), int(x.k)))
        exp_dict = {}
        
        for exp in sub_experiments:
            try:
                exp_dict.get(('k', exp.k)).append(exp)
            except:
                exp_dict[('k', exp.k)] = [exp]
                
        subplot_dimens = (2, len(exp_dict.keys()))
        current_row = 0
        fig, axes = plt.subplots(*subplot_dimens, figsize=(15,10))
        fig.suptitle(f'Trickle (imin={filter_params["imin"]}, imax={i}) in a 15x15 network, failure mode={filter_params["t"]}, max failures={filter_params["n"]}')
        
        for key in exp_dict.keys():
            control_params = {
                'd': None, 
                'k': key[1],
                'imin': imin,
                't': None,
                'imax': i, 
                'n': None # n is the X axis here
            }
            
            control_exp = filter_experiments(control.keys(), Experiment(**control_params)).pop()
            control_data = control[control_exp]
            control_msgs_sent = list(map(lambda x: int(x['messages']), control_data))
            
            control_msgs_avg = np.mean(filter_anomalies(control_msgs_sent))
            
            x_axis = list(map(lambda x: int(x.d), exp_dict[key]))
            figs = axes[current_row]
            
            coverage_pts = []
            msgs_pts = []
            
            for exp in exp_dict[key]:
                msgs = list(map(lambda x: int(x['messages']), experiments[exp]))
                coverage = list(map(lambda x: float(x['coverage']), experiments[exp]))
                
                msgs_pts.append((
                    np.mean(filter_anomalies(msgs)), 
                    np.std(msgs)
                ))
                coverage_pts.append((
                    np.mean(filter_anomalies(coverage)), 
                    np.std(coverage)
                ))
                
            msgs_ax = figs[0]
            msgs_ax.set_xticks(x_axis)
            msgs_ax.grid(linestyle='--')
            msgs_ax.set_xlim(0.5, 15.5)
            msgs_ax.set_ylim(750,1400)
            msgs_ax.set_title(f'Total messages sent for {key[0]}={key[1]}')
            msgs_ax.set_ylabel('Total messages sent')
            msgs_ax.set_xlabel('Mote crash time (s)')
            msgs_ax.errorbar(x_axis, *list(map(list, zip(*msgs_pts))), 
                             marker='^', color='royalblue', ecolor='orange', capsize=4)
            msgs_ax.axhline(control_msgs_avg, color='orangered', linestyle='--')
            msgs_ax.text(4.15, control_msgs_avg, '0 fails avg', horizontalalignment='left', verticalalignment='center',
                        color='red')
            
            coverage_ax = figs[1]
            coverage_ax.set_xticks(x_axis)
            coverage_ax.grid(linestyle='--')
            coverage_ax.set_xlim(0.5, 15.5)
            coverage_ax.set_ylim(65, 112)
            coverage_ax.set_ylabel('Message coverage (%)')
            coverage_ax.set_xlabel('Mote crash time (s)')
            coverage_ax.set_title(f'Message coverage for {key[0]}={key[1]}')
            coverage_ax.errorbar(x_axis, *list(map(list, zip(*coverage_pts))), 
                             marker='^', color='royalblue', ecolor='orange', capsize=4)
            coverage_ax.axhline(100, color='orangered', linestyle='--')
            coverage_ax.text(4.15, 100, '0 fails avg', horizontalalignment='left', verticalalignment='center',
                        color='red')
            current_row = current_row + 1
        
        fig.subplots_adjust(top=0.92, bottom=0.08, left=0.06, right=0.9, hspace=0.25,
                wspace=0.35)
        fig.savefig(format='png', fname=f'graphs/mote_delay/trickle-15x15-{imin}-{i}-{filter_params["n"]}-{filter_params["t"]}-delay.png', dpi=150)
        fig.clf()
        
  
recovery_delay(imin=4)
recovery_delay(imin=8)
recovery_delay(imin=16)
recovery_delay(imin=32)

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

In [9]:
def max_failures(imin=16):
    filter_params = {
        'd': 15, 
        'k': (3, 4),
        'imin': imin,
        't': 'random',
        'imax': (11, ), 
        'n': None # n is the X axis here
    }
    all_experiments = filter_experiments(experiments, Experiment(**filter_params))
    
    if not os.path.exists('graphs/max_fail/'):
        os.makedirs('graphs/max_fail')
    
    for i in filter_params['imax']:
        sub_filter = Experiment(d=None, k=None, imin=None, t=None, imax=i, n=None)
        sub_experiments = list(filter_experiments(all_experiments, sub_filter))
        sub_experiments = sorted(sub_experiments, key=lambda x: (int(x.n), int(x.k)))
        exp_dict = {}
        
        for exp in sub_experiments:
            try:
                exp_dict.get(('k', exp.k)).append(exp)
            except:
                exp_dict[('k', exp.k)] = [exp]
                
        subplot_dimens = (2, len(exp_dict.keys()))
        current_row = 0
        fig, axes = plt.subplots(*subplot_dimens, figsize=(15,10))
        fig.suptitle(f'Trickle (imin={filter_params["imin"]}, imax={i}) in a 15x15 network, failure mode={filter_params["t"]}, mote delay={filter_params["d"]}s')
        
        for key in exp_dict.keys():
            control_params = {
                'd': None, 
                'k': key[1],
                'imin': imin,
                't': None,
                'imax': i, 
                'n': None # n is the X axis here
            }
            
            control_exp = filter_experiments(control.keys(), Experiment(**control_params)).pop()
            control_data = control[control_exp]
            control_msgs_sent = list(map(lambda x: int(x['messages']), control_data))
            
            control_msgs_avg = np.mean(filter_anomalies(control_msgs_sent))
            
            x_axis = list(map(lambda x: int(x.n), exp_dict[key]))
            figs = axes[current_row]
            
            coverage_pts = []
            msgs_pts = []
            
            for exp in exp_dict[key]:
                msgs = list(map(lambda x: int(x['messages']), experiments[exp]))
                coverage = list(map(lambda x: float(x['coverage']), experiments[exp]))
                
                msgs_pts.append((
                    np.mean(filter_anomalies(msgs)), 
                    np.std(msgs)
                ))
                coverage_pts.append((
                    np.mean(filter_anomalies(coverage)), 
                    np.std(coverage)
                ))
                
            msgs_ax = figs[0]
            msgs_ax.set_xticks(x_axis)
            msgs_ax.grid(linestyle=':')
            msgs_ax.set_xlim(0.5, 15.5)
            msgs_ax.set_ylim(750,1350)
            msgs_ax.set_title(f'Total messages sent for {key[0]}={key[1]}')
            msgs_ax.set_ylabel('Total messages sent')
            msgs_ax.set_xlabel('Maximum simultaneous failures (motes)')
            msgs_ax.errorbar(x_axis, *list(map(list, zip(*msgs_pts))), 
                             marker='^', color='royalblue', ecolor='cornflowerblue', capsize=4)
            msgs_ax.axhline(control_msgs_avg, color='forestgreen', linestyle='--')
            msgs_ax.text(15.8, control_msgs_avg, '0 fails avg', horizontalalignment='left', verticalalignment='center',
                        color='forestgreen')
            
            coverage_ax = figs[1]
            coverage_ax.set_xticks(x_axis)
            coverage_ax.grid(linestyle=':')
            coverage_ax.set_xlim(0.5, 15.5)
            coverage_ax.set_ylim(60, 112)
            coverage_ax.set_ylabel('Message coverage (%)')
            coverage_ax.set_xlabel('Maximum simultaneous failures (motes)')
            coverage_ax.set_title(f'Message coverage for {key[0]}={key[1]}')
            coverage_ax.errorbar(x_axis, *list(map(list, zip(*coverage_pts))), 
                             marker='^', color='royalblue', ecolor='cornflowerblue', capsize=4)
            coverage_ax.axhline(100, color='forestgreen', linestyle='--')
            coverage_ax.text(15.8, 100, '0 fails avg', horizontalalignment='left', verticalalignment='center',
                        color='forestgreen')
            current_row = current_row + 1
        
        fig.subplots_adjust(top=0.92, bottom=0.08, left=0.06, right=0.9, hspace=0.25,
                wspace=0.35)
        fig.savefig(format='png', fname=f'graphs/max_fail/trickle-15x15-{imin}-{i}-{filter_params["d"]}-{filter_params["t"]}-fails.png', dpi=150)
        fig.clf()
        
        
max_failures(imin=4)
max_failures(imin=8)
max_failures(imin=16)
max_failures(imin=32)

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>